# Projet 10 : Réalisez une application de recommandation de contenu

## Contexte

Vous êtes le CTO et cofondateur de la start-up My Content qui veut encourager la lecture en recommandant des contenus pertinents pour ses utilisateurs. Vous êtes en pleine construction d’un premier MVP (Minimum Viable Product) qui prendra la forme d’une application.

Dans un premier temps, votre start-up souhaite tester une solution de recommandation d’articles et de livres à des particuliers en utilisant des données disponibles en ligne puisque vous n'en avez pas encore.

Votre mission est de développer une première version de votre système de recommandation d'une sélection de cinq articles, sous forme d’Azure Functions, tout en prenant en compte l'ajout de nouveaux articles et utilisateurs.

Puis de réaliser une application simple de gestion du système de recommandation (interface d’affichage d’une liste d’id utilisateurs, d’appel Azure functions pour l’id choisi, et d’affichage des 5 articles recommandés)

## Contenu des données du jeu de données News Portal User Interactions by Globo.com :

Le jeu de données provient du portail d’actualité brésilien Globo. Il traite des clics utilisateurs/articles.

Il contient 3 fichiers et un dossier zip :
- clicks.zip : contient les données d’interaction utilisateur → article (clics, horodatage, etc.).

- clicks_sample.csv : contient un échantillon des données d’interaction utilisateur → article (clics, horodatage, etc.).

- articles_metadata.csv : les métadonnées des articles (titre, catégorie, etc.).

- articles_embeddings.pickle : embeddings pré-calculés des articles.


## Librairies utilisées avec le langage Python :

In [ ]:
!pip install numpy==1.26.4
!pip install scikit-surprise
!pip install implicit

  Using cached implicit-0.7.2.tar.gz (70 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for implicit: filename=implicit-0.7.2-cp312-cp312-linux_x86_64.whl size=10855794 sha256=1f1873b4c9eecd855ba5539f879f465d54d8d23edaedcf618ad30e85cfb44f43
  Stored in directory: /root/.cache/pip/wheels/b2/00/4f/9ff8af07a0a53ac6007ea5d739da19cfe147a2df542b6899f8
Successfully built implicit


In [ ]:
# Importation des librairies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix, coo_matrix
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split, cross_validate
from surprise import SVD, accuracy
import implicit
import tqdm
import os
import warnings
warnings.filterwarnings('ignore')

## Données :

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Importation des données :
# data = pd.read_csv('C:/tutorial-env/OCR/Projet10/data/df_post_fe.csv',encoding='latin',delimiter=",")
data = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/P10/df_post_fe.csv',encoding='latin',delimiter=",")
data.head()

,user_id,session_id,session_start,session_size,article_id,click_timestamp,article_category_id,article_words_count,article_created_at,nb_session_per_user,nb_article_per_user,nb_category_article_per_user,avg_wordcount_per_user,nb_article_per_session,nb_category_article_per_session,avg_wordcount_per_session
0,0,1506825423271737,2017-10-01 02:37:03,2,157541,2017-10-01 03:00:28.020,281,280,2017-09-30 19:41:58,4,8,6,207.000000,2,2,253.0
1,0,1506825423271737,2017-10-01 02:37:03,2,68866,2017-10-01 03:00:58.020,136,226,2017-10-01 00:08:02,4,8,6,207.000000,2,2,253.0
2,1,1506825426267738,2017-10-01 02:37:06,2,235840,2017-10-01 03:03:37.951,375,159,2017-09-30 21:43:59,6,12,9,212.083333,2,2,182.5
3,1,1506825426267738,2017-10-01 02:37:06,2,96663,2017-10-01 03:04:07.951,209,206,2017-09-30 16:13:45,6,12,9,212.083333,2,2,182.5
4,2,1506825435299739,2017-10-01 02:37:15,2,119592,2017-10-01 03:04:50.575,247,239,2017-09-30 15:11:56,2,4,3,200.500000,2,2,227.5


In [ ]:
data_articles = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/P10/articles_metadata.csv',encoding='latin',delimiter=",")
data_articles.head()
print(data_articles.shape)

(364047, 5)


In [ ]:
with open('/content/drive/MyDrive/Colab_Notebooks/P10/articles_embeddings.pickle', 'rb') as f:
    embeddings = pickle.load(f)
df_embeddings= pd.DataFrame(embeddings)
df_embeddings.head()

,0,1,2,3,4,5,6,7,8,9,...,240,241,242,243,244,245,246,247,248,249
0,-0.161183,-0.957233,-0.137944,0.050855,0.830055,0.901365,-0.335148,-0.559561,-0.500603,0.165183,...,0.321248,0.313999,0.636412,0.169179,0.540524,-0.813182,0.286870,-0.231686,0.597416,0.409623
1,-0.523216,-0.974058,0.738608,0.155234,0.626294,0.485297,-0.715657,-0.897996,-0.359747,0.398246,...,-0.487843,0.823124,0.412688,-0.338654,0.320787,0.588643,-0.594137,0.182828,0.397090,-0.834364
2,-0.619619,-0.972960,-0.207360,-0.128861,0.044748,-0.387535,-0.730477,-0.066126,-0.754899,-0.242004,...,0.454756,0.473184,0.377866,-0.863887,-0.383365,0.137721,-0.810877,-0.447580,0.805932,-0.285284
3,-0.740843,-0.975749,0.391698,0.641738,-0.268645,0.191745,-0.825593,-0.710591,-0.040099,-0.110514,...,0.271535,0.036040,0.480029,-0.763173,0.022627,0.565165,-0.910286,-0.537838,0.243541,-0.885329
4,-0.279052,-0.972315,0.685374,0.113056,0.238315,0.271913,-0.568816,0.341194,-0.600554,-0.125644,...,0.238286,0.809268,0.427521,-0.615932,-0.503697,0.614450,-0.917760,-0.424061,0.185484,-0.580292


### Nettoyage

Pour réduire le bruit dans les données, accélérer l'entraînement (moins d’utilisateurs/articles inutiles) et améliorer la pertinence des recommandations,  on va enlever les lignes des utilisateurs qui ont consultés moins de 3 articles et celles concernant les articles qui ont été consultés mois de 3 fois.
En effet, les embeddings / scores sont mieux appris sur des profils “riches”.

In [ ]:
# Nombre d'interactions par utilisateur
user_counts = data.groupby('user_id').size()
# Garde uniquement les utilisateurs avec au moins 3 interactions
valid_users = user_counts[user_counts >= 3].index

# Nombre d'interactions par article
item_counts = data.groupby('article_id').size()
# Garde uniquement les articles avec au moins 3 interactions
valid_items = item_counts[item_counts >= 3].index

# Filtrage du dataset
df_filtered = data[data['user_id'].isin(valid_users) & data['article_id'].isin(valid_items)]

print(f"Avant filtrage : {data.shape[0]} clics")
print(f"Après filtrage : {df_filtered.shape[0]} clics")
print(f"Nb utilisateurs : {df_filtered['user_id'].nunique()}, Nb articles : {df_filtered['article_id'].nunique()}")

Avant filtrage : 2965757 clics
Après filtrage : 2735228 clics
Nb utilisateurs : 220392, Nb articles : 16369


In [ ]:
df_filtered.to_csv("/content/drive/MyDrive/Colab_Notebooks/P10/df_filtered.csv", index=False)

In [ ]:
df_clik_azure=df_filtered[['user_id', 'article_id']]
df_clik_azure.to_csv("/content/drive/MyDrive/Colab_Notebooks/P10/df_clik_azure.csv", index=False)

## Modélisation :

### Content-based

- Prendre les articles déjà consultés (clic) par l’utilisateur

- Calculer la similarité avec les embeddings des autres articles

- Par utilisateur : proposer les plus similaires non encore lus / Par article : les plus similaires

In [ ]:
# Normalisation
scaler = StandardScaler()
df_embeddings= scaler.fit_transform(df_embeddings)

In [ ]:
def recommend_for_user(user_id, interactions_df, df_meta, embeddings, top_n=5):
    click_idx = []
    # Articles cliqués par l'utilisateur
    click_ids = interactions_df.loc[interactions_df['user_id'] == user_id, 'article_id'].unique()
    if len(click_ids) == 0:
        return []

    # Récupérer les embeddings correspondants
    for id in click_ids:
      click_idx.append(df_meta[df_meta['article_id']==int(id)].index)

    # Faire la moyenne vectorielle de ces articles (représentation moyenne des goûts de l’utilisateur)
    user_vector = embeddings[click_idx].mean(axis=0).reshape(1, -1)

    # Similarité cosinus entre le vecteur moyen utilisateur et tous les articles
    sims = cosine_similarity(user_vector, embeddings)[0]

    # Exclure les articles déjà lus
    mask = ~df_meta['article_id'].isin(click_ids)
    sims_masked = np.where(mask, sims, -np.inf)

    # Garder seulement le nombre déterminé
    top_idx = np.argsort(sims_masked)[::-1][:top_n]
    return df_meta.iloc[top_idx][['article_id']].assign(score=sims[top_idx])

In [ ]:
# Test d'une recommandation par utilisateur
recommendations_user = recommend_for_user(user_id=10, interactions_df=df_filtered, df_meta=data_articles, embeddings=df_embeddings, top_n=5)
print(recommendations_user)

        article_id     score
202819      202819  0.785059
206490      206490  0.779459
205844      205844  0.779189
202232      202232  0.762410
202119      202119  0.755578


In [ ]:
def recommend_similar_articles(article_id, df_meta, embeddings, top_n=5):
    # Trouver l'index correspondant à l'article
    idx = df_meta.index[df_meta['article_id'] == article_id]
    if len(idx) == 0:
        return []
    idx = idx[0]

    # Embedding de l'article de référence
    target_vector = embeddings[idx].reshape(1, -1)

    # Similarité cosinus avec tous les autres
    sims = cosine_similarity(target_vector, embeddings)[0]

    # Trier par similarité décroissante en excluant l'article lui-même
    similar_idx = sims.argsort()[::-1]
    similar_idx = similar_idx[similar_idx != idx][:top_n]

    # Retourner les IDs + scores
    return df_meta.iloc[similar_idx][['article_id']].assign(score=sims[similar_idx])

In [ ]:
# Test d'une recommandation par article
recommendations = recommend_similar_articles(article_id=68866, df_meta=data_articles, embeddings=df_embeddings, top_n=5)
print(recommendations)

        article_id     score
358037      358037  0.788020
65360        65360  0.771053
159170      159170  0.764183
152054      152054  0.757607
160832      160832  0.755137


In [ ]:
#  Sauvegarde des 20 embeddings les plus proches les uns des autres de la matrice de cosimiliarité
N = embeddings.shape[0]
K = 20  # nombre de voisins à garder

topk_dict = {}

batch_size = 500  # pour ne pas saturer la RAM
for start in tqdm.tqdm(range(0, N, batch_size)):
    end = min(start + batch_size, N)
    sims = cosine_similarity(embeddings[start:end], embeddings)
    # Pour chaque article dans le batch, extraire top K
    for i, row in enumerate(sims):
        idx_sorted = np.argsort(row)[::-1]  # décroissant
        top_indices = idx_sorted[1:K+1]  # on ignore l'article lui-même (0)
        topk_dict[start + i] = top_indices.tolist()

# Sauvegarde en pickle
with open("/content/drive/MyDrive/Colab_Notebooks/P10/topk_neighbors.pkl", "wb") as f:
    pickle.dump(topk_dict, f)


100%|██████████| 729/729 [1:43:17<00:00,  8.50s/it]


### Collaborative filtering avec librairie Surprise

- Preprocessing : vers un dataset avec (user, item, rating). Normalement Surprise est fait pour un rating explicite mais ici, rating = 1 si clic (rating implicite).

- Entrainement avec un modèle SVD (Décomposition en Valeurs Singulières)

- Proposer ceux que d'autres utilisateurs avec le même profil ont consulté

In [ ]:
# Préprocessing
df_surprise = df_filtered[['user_id', 'article_id']].copy()
df_surprise['rating'] = 1.0  # feedback implicite

# Format adéquat pour Surprise
reader = Reader(rating_scale=(0, 1))
data_surprise = Dataset.load_from_df(df_surprise[['user_id', 'article_id', 'rating']], reader)


In [ ]:
# Entrainement via le modèle SVD
algo = SVD()
# Validation croisée pour évaluer le modèle
cross_validate(algo, data_surprise, measures=["RMSE", "MAE"], cv=3, verbose=True)
#  Découpage en jeu d'entrainement et de validation
trainset, testset = train_test_split(data_surprise, test_size=0.2)
# Entrainement
algo.fit(trainset)

Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.0252  0.0252  0.0252  0.0252  0.0000  
MAE (testset)     0.0086  0.0086  0.0086  0.0086  0.0000  
Fit time          47.82   50.00   49.28   49.03   0.91    
Test time         8.31    7.36    6.84    7.50    0.61    


In [ ]:
# Prédictions
predictions = algo.test(testset)

In [ ]:
# Évaluer la précision globale
accuracy.rmse(predictions)

# Recommandations pour un utilisateur précis
user_id = str(10)  # Librairie Surprise attend des str
all_items = df_surprise["article_id"].unique()

# Articles déjà lus
items_read = df_surprise[df_surprise["user_id"] == int(user_id)]["article_id"].unique()

# Articles à recommander = ceux jamais lus
items_to_pred = [iid for iid in all_items if iid not in items_read]

# Prédire la note pour chaque item non lu
preds = [algo.predict(user_id, iid) for iid in items_to_pred]

# Trier par rating prédit
top_n = sorted(preds, key=lambda x: x.est, reverse=True)[:5]
recommandations = [pred.iid for pred in top_n]

print("Articles recommandés :", recommandations)

RMSE: 0.0238
Articles recommandés : ['157541', '68866', '48915', '284847', '332114']


La librairie Surprise est faite pour des notations explicites et n'est donc pas vraiment pertinent pour notre cas actuel.

### Collaborative filtering avec librairie Implicit

- Preprocessing : vers une matrice sparse de type (user, item) avec les interactions (clics).

- Entrainement avec un modèle ALS (Alternating Least Squares)

- Proposer ceux que d'autres utilisateurs avec le même profil ont consulté

In [ ]:
# Mappings
user_cats = pd.Categorical(df_filtered['user_id'])
item_cats = pd.Categorical(df_filtered['article_id'])

# Sauvegard des mappings
idx_to_user = dict(enumerate(user_cats.categories))
user_to_idx = {v: k for k, v in idx_to_user.items()}

idx_to_item = dict(enumerate(item_cats.categories))
item_to_idx = {v: k for k, v in idx_to_item.items()}

# Construire matrice item-user
rows = item_cats.codes
cols = user_cats.codes
interactions = csr_matrix((np.ones(len(df_filtered)), (rows, cols)))

# Entraînement
model = implicit.als.AlternatingLeastSquares(factors=50,
                                regularization=0.01,
                                dtype=np.float64,
                                iterations=50)
model.fit(interactions.T)


  0%|          | 0/50 [00:00<?, ?it/s]

[124350, 234269, 225019, 48403, 31520]


In [ ]:
# Test d'une recommandation par utilisateur
userid = 10
user_idx = user_to_idx[userid]

# Recommandation
ids, scores = model.recommend(user_idx, interactions.T.tocsr()[user_idx], N=5)

# Conversion des indices internes en ids réels
recommended_article_ids = [idx_to_item[i] for i in ids]
print(recommended_article_ids)

[124350, 234269, 225019, 48403, 31520]


In [ ]:
# Test d'une recommandation par article
article_id = 68866

# Convertir en index interne
item_idx = item_to_idx[article_id]

# Rechercher les articles similaires
ids, scores = model.similar_items(item_idx, N=6)  # N=6 pour inclure l'article lui-même

# Exclure l'article lui-même car renvoyé en premier
ids = ids[1:]
scores = scores[1:]

# Conversion indices internes en id réels
similar_article_ids = [idx_to_item[i] for i in ids]

# Affichage
for aid, s in zip(similar_article_ids, scores):
    print(f"Article similaire {aid} — score {s:.4f}")

Article similaire 108854 — score 0.9638
Article similaire 158866 — score 0.9374
Article similaire 156543 — score 0.9373
Article similaire 338351 — score 0.9362
Article similaire 284664 — score 0.9233


## Evaluation :

De façon classique, on sépare les jeu de données temporellement pour pouvoir entraîner sur une partie ancienne le modèle et de tester les prédictions sur la partie récente.

Cependant, le laps de temps très court de notre jeu de données (environ 15 jours) ne peut pas nous permettre de réaliser cette évaluation actuellement.


Dans notre projet, on pourrait imaginer faire un une évaluation en conditions réelles ou A/B testing.

On séparerait les utilisateurs en deux groupes, randomisés aléatoirement :

- Groupe A → sans recommandation du modèle, juste les 5 articles les plus consultés

- Groupe B → reçoit les recommandations du modèle

Ensuite, les performances réelles sont comparées selon des métriques terrain : taux de recommandations consultés, temps passé, nombre d’articles consultés,...

## Déploiement :

Le déploiement imposé dans une Azure Fonction HTTP trigger ne permet pas de pouvoir calculer une cosimilarité, le traitement est trop lourd pour ce type de fonction.

La prédiction des articles recommandés pour chaque utilisateur sera donc effectué en amont.

In [ ]:
# Liste de tous les user_id
user_ids = df_filtered['user_id'].unique()
user_ids_list = user_ids.tolist()

In [11]:
BATCH_SIZE = 500
output_path = '/content/drive/MyDrive/Colab_Notebooks/P10/precomputed_recos.csv'

# Charger les user_ids restants si le fichier existe déjà
if os.path.exists(output_path):
    recos_df = pd.read_csv(output_path)
    processed_users = set(recos_df['user_id'])
else:
    recos_df = pd.DataFrame()
    processed_users = set()

remaining_users = [u for u in user_ids_list if u not in processed_users]
print(f"{len(remaining_users)} utilisateurs restants à traiter.")

for i in range(0, len(remaining_users), BATCH_SIZE):
    batch_users = remaining_users[i:i+BATCH_SIZE]
    batch_results = []

    for user in batch_users:
        recos = recommend_for_user(
            user_id=user,
            interactions_df=df_filtered,
            df_meta=data_articles,
            embeddings=df_embeddings,
            top_n=5
        )

        recos_str = ",".join(recos['article_id'].astype(str).tolist())
        batch_results.append({"user_id": user, "recommended_articles": recos_str})

    # Sauvegarde progressive dans le CSV
    batch_df = pd.DataFrame(batch_results)
    if os.path.exists(output_path):
        batch_df.to_csv(output_path, mode='a', header=False, index=False)
    else:
        batch_df.to_csv(output_path, index=False)

    print(f"✅ Batch {i//BATCH_SIZE+1} traité ({len(batch_users)} utilisateurs)")

39392 utilisateurs restants à traiter.
✅ Batch 1 traité (500 utilisateurs)
✅ Batch 2 traité (500 utilisateurs)
✅ Batch 3 traité (500 utilisateurs)
✅ Batch 4 traité (500 utilisateurs)
✅ Batch 5 traité (500 utilisateurs)
✅ Batch 6 traité (500 utilisateurs)
✅ Batch 7 traité (500 utilisateurs)
✅ Batch 8 traité (500 utilisateurs)
✅ Batch 9 traité (500 utilisateurs)
✅ Batch 10 traité (500 utilisateurs)
✅ Batch 11 traité (500 utilisateurs)
✅ Batch 12 traité (500 utilisateurs)
✅ Batch 13 traité (500 utilisateurs)
✅ Batch 14 traité (500 utilisateurs)
✅ Batch 15 traité (500 utilisateurs)
✅ Batch 16 traité (500 utilisateurs)
✅ Batch 17 traité (500 utilisateurs)
✅ Batch 18 traité (500 utilisateurs)
✅ Batch 19 traité (500 utilisateurs)
✅ Batch 20 traité (500 utilisateurs)
✅ Batch 21 traité (500 utilisateurs)
✅ Batch 22 traité (500 utilisateurs)
✅ Batch 23 traité (500 utilisateurs)
✅ Batch 24 traité (500 utilisateurs)
✅ Batch 25 traité (500 utilisateurs)
✅ Batch 26 traité (500 utilisateurs)
✅ Batch 